In [1]:
from google.colab import drive
drive.mount('/content/gdrive')  #mounting


Mounted at /content/gdrive


In [2]:
%cd '/content/gdrive/MyDrive/keras_translation'

/content/gdrive/MyDrive/keras_translation


In [3]:
!git clone 'https://github.com/divamgupta/attention-translation-keras.git'

Cloning into 'attention-translation-keras'...
remote: Enumerating objects: 15, done.
remote: Total 15 (delta 0), reused 0 (delta 0), pack-reused 15
Unpacking objects: 100% (15/15), done.


In [5]:
mkdir data

In [6]:
cd data


/content/gdrive/MyDrive/keras_translation/data


In [8]:
!wget "http://www.cfilt.iitb.ac.in/iitb_parallel/iitb_corpus_download/parallel.tgz"


--2020-12-23 11:28:52--  http://www.cfilt.iitb.ac.in/iitb_parallel/iitb_corpus_download/parallel.tgz
Resolving www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)... 103.21.127.130
Connecting to www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)|103.21.127.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100235015 (96M) [application/x-gzip]
Saving to: ‘parallel.tgz’

parallel.tgz        100%[===================>]  95.59M  5.43MB/s    in 38s     

2020-12-23 11:29:30 (2.50 MB/s) - ‘parallel.tgz’ saved [100235015/100235015]



In [10]:
!tar -xf "parallel.tgz"


In [11]:
cd ..

/content/gdrive/My Drive/keras_translation


In [13]:
%cd '/content/gdrive/MyDrive/keras_translation/attention-translation-keras'

/content/gdrive/MyDrive/keras_translation/attention-translation-keras


In [17]:
import h5py
import json
import argparse


from utils import getSentencesMat , Vocabulary


#parser = argparse.ArgumentParser(description='NMT Keras')
#parser.add_argument('--text_A', type=str,  help='Corpus text file of language A')
#parser.add_argument('--text_B', type=str,  help='Corpus text file of language B')
#parser.add_argument('--out_file', type=str,  default="./data/nmt_hi_en_prepped.h5" , help='Output HDF5 file name')
#args = parser.parse_args()


eng_sents = (open('/content/gdrive/MyDrive/keras_translation/data/parallel/IITB.en-hi.en').read()).split("\n")[:-1]
hi_sents = (open('/content/gdrive/MyDrive/keras_translation/data/parallel/IITB.en-hi.hi').read()).split("\n")[:-1]


en_vocab = Vocabulary()
hi_vocab = Vocabulary()







In [18]:
for ens in eng_sents:
	en_vocab.add_words( ens.split(' ') )
	
for his in hi_sents:
	hi_vocab.add_words( his.split(' ') )

en_vocab.keepTopK(40000)
hi_vocab.keepTopK(40000)


eng_sent_mat = getSentencesMat( eng_sents  ,en_vocab , startEndTokens=True , tokenizer_fn=lambda x:x.split(' ') , maxSentenceL=100 )
hin_sent_mat = getSentencesMat( hi_sents  ,hi_vocab , startEndTokens=True , tokenizer_fn=lambda x:x.split(' ') , maxSentenceL=100 )

In [21]:
ls

model.py  predict.py  prep_data.py  __pycache__/  README.md  train.py  utils.py


In [22]:

f = h5py.File('/content/gdrive/MyDrive/keras_translation/data/nmt_hi_en_prepped.h5' , "w")

f.create_dataset("hi_vocab",   data=json.dumps(hi_vocab.getDicts()) )
f.create_dataset("en_vocab",   data=json.dumps(en_vocab.getDicts()) )

f.create_dataset("en_sents",   data=json.dumps( eng_sents ) )
f.create_dataset("hi_sents",   data=json.dumps( hi_sents ) )

f.create_dataset('eng_sent_mat' , data=eng_sent_mat )
f.create_dataset('hin_sent_mat' , data=hin_sent_mat )


f.close()

In [23]:
%cd '/content/gdrive/MyDrive/keras_translation/attention-translation-keras'

/content/gdrive/MyDrive/keras_translation/attention-translation-keras


In [24]:
mkdir weights


In [31]:
!python train.py --dataset="/content/gdrive/MyDrive/keras_translation/data/nmt_hi_en_prepped.h5" --weights_path="/weights/KerasAttentionNMT_1.h5"


2020-12-23 12:35:41.219027: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2020-12-23 12:36:05.775413: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-23 12:36:05.782727: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-23 12:36:05.795093: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-12-23 12:36:05.795163: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dcd8a33b8878): /proc/driver/nvidia/version does not exist
2020-12-23 12:36:05.795829: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/traini

In [ ]:
!python predict.py --dataset="/content/gdrive/MyDrive/keras_translation/data/nmt_hi_en_prepped.h5" --weights_path="/weights/KerasAttentionNMT_1.h5"